In [205]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from bokeh.io import output_notebook

In [206]:
df = pd.read_csv(r"2017 Terms and Incumbents Data Analysis.csv")
df.columns = df.columns.str.replace(' ', '_')
df.rename(columns= {'﻿Employee_Number' : 'Employee_Number', '﻿Date':'Date'}, inplace = True)

In [207]:
df.sort_values(by=['Employee_Number', 'Date'], inplace=True)
df.drop_duplicates(subset = ['Employee_Number'], keep = 'last', inplace = True)

In [222]:
df['Termination_Flag'] = 0
df.loc[df.Employee_Actual_Termination_Date.isnull(), 'Termination_Flag'] = 0
df.loc[~df.Employee_Actual_Termination_Date.isnull(), 'Termination_Flag'] = 1

df.loc[df.Termination_Flag ==1,'nb_months'] = (pd.to_datetime(df.Date) - pd.to_datetime(df.Employee_Original_Hire_Date))/ np.timedelta64(1, 'Y')
df.loc[~df.Termination_Flag==0,'nb_months'] = (date.today() - pd.to_datetime(df.Employee_Original_Hire_Date))/ np.timedelta64(1, 'Y')

In [223]:
T = df['nb_months']
E = df['Termination_Flag']
output_notebook()

Loading BokehJS ...

In [224]:
def kmf_country(country= 'India'):
    
    from lifelines import KaplanMeierFitter
    import numpy as np
    from bokeh.layouts import gridplot
    from bokeh.plotting import figure, show, output_file
    from bokeh.palettes import Spectral11, Category10
    from bokeh.models import HoverTool
    
    country_df = df[df.Payroll_Country.isin([country])]
    
    sbu_list = list(country_df['SBU'].unique())
    kmf_indx_list = [] 
    kmf_func_list = []
    T_dict = {}
    E_dict = {}
    K_dict = {}
    
    for _ in sbu_list:
        T_dict[_] = country_df[country_df.SBU.isin([_])]['nb_months']
        E_dict[_] = country_df[country_df.SBU.isin([_])]['Termination_Flag']
        K_dict[_] = KaplanMeierFitter()
        K_dict[_].fit(T_dict[_], event_observed=E_dict[_])
        kmf_indx_list.append(K_dict[_].survival_function_.index.tolist())
        kmf_func_list.append(K_dict[_].survival_function_.KM_estimate.tolist())
        

    numlines=len(sbu_list)
    mypalette=Category10[10][0:numlines]
    
    p = figure()
    
    for (colr, leg, x, y ) in zip(mypalette, sbu_list, kmf_indx_list, kmf_func_list):
        my_plot = p.line(x, y, color= colr, legend= leg)
    return show(p)

In [227]:
kmf_country('India')